In [28]:
import numpy as np
import re
import ast
import subprocess
import os, csv
from codes.utils.file import create_file_abs, create_parent_dir, dir_exists_abs, file_exists_absolute, join_dir, file_exists_current, file_remove_current
from codes.parameters import ASTRAL, NUMPY_EXTENSION, QUARTETS, SCRIPT_FOLDER, STATS_FOLDER, TRUE_FOLDER, WQFM, \
    COMPLETE, GENE_TREE, QUARTETS, INCOMPLETE, IMPUTED_LIST, OUTPUT_FOLDER, NUMPY_ARRAY, REMOVED_TAXA_RANGE, INPUT_FOLDER, \
    QUARTETS_EXTENSION, MODES_LIST, WORKING_FOLDER, METHODS, TERRACE_OUTPUT_FOLDER

In [29]:
# from IPython import get_ipython

In [30]:
terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
consensus_dir = join_dir(terrace_dir, "consensus")
!rm -rf "$consensus_dir"
!mkdir "$consensus_dir"

if not dir_exists_abs(terrace_dir):
    print("Directory does not exist: {}".format(terrace_dir))
    exit(1)

In [31]:
# get consensus trees
for terrace_file in os.listdir(terrace_dir):
    file_path = join_dir(terrace_dir, terrace_file)
    if file_path.endswith(".txt"):
        print("Processing file: {}".format(file_path))
        terrace_file_name = terrace_file.split(".")[0]
        !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name"
        # shell_script_path = join_dir("./bin/raxml-ng")
        # output_path = join_dir(consensus_dir, terrace_file_name)
        # result = subprocess.run([shell_script_path, "--consense MRE", "--tree", file_path, "--prefix", output_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        # print(result.stdout)

Processing file: terrace_output/14-17_12_gt_species.txt

RAxML-NG v. 1.1.0-master released on 04.08.2022 by The Exelixis Lab.
Developed by: Alexey M. Kozlov and Alexandros Stamatakis.
Contributors: Diego Darriba, Tomas Flouri, Benoit Morel, Sarah Lutteropp, Ben Bettisworth.
Latest version: https://github.com/amkozlov/raxml-ng
Questions/problems/suggestions? Please visit: https://groups.google.com/forum/#!forum/raxml

System: Apple M1, 8 cores, 8 GB RAM

RAxML-NG was called at 31-Jan-2023 11:26:36 as follows:

./bin/raxml-ng --consense MRE --tree terrace_output/14-17_12_gt_species.txt --prefix terrace_output/consensus/14-17_12_gt_species

Analysis options:
  run mode: Build consensus tree (MRE)
  start tree(s): user
  random seed: 1675142796
  SIMD kernels: SSE3
  parallelization: coarse-grained (auto), PTHREADS (auto)

Reading input trees from file: terrace_output/14-17_12_gt_species.txt
Loaded 2001 trees with 37 taxa.

MRE consensus tree saved to: /Users/sayemhasan/Desktop/Thesis/Terr

In [32]:
def remove_numbers_from_consensus_tree(file_path):
    file = open(file_path, "r")
    lines = file.readlines()
    line = lines[0]
    line = re.sub(r'[0-9]', '', line)
    # print(line)
    file = open(file_path, 'w')
    file.writelines(line)
    file.close()

In [33]:
# get RF distance
def get_RF_distance(true_tree, file_abs):
    result = ast.literal_eval(subprocess.check_output(['getFpFn.py', '-t', true_tree, '-e', file_abs], encoding='UTF-8'))
    assert result[0] == result[1] == result[2]
    dist = str(result[0])
    # print( dist)
    return dist

In [34]:
true_tree = join_dir(TRUE_FOLDER, 'true-species.out.tree')

In [35]:
print("Removing numbers from consensus trees")
CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
csv_out_list = []
for consensus_file in os.listdir(consensus_dir):
    if consensus_file.endswith(CONSENSUS_TREE_EXTENSION):
        file_path = join_dir(consensus_dir, consensus_file)
        print("Processing file: {}".format(file_path))
        remove_numbers_from_consensus_tree(file_path)
        dist = get_RF_distance(true_tree, file_path)
        print("RF distance: {}".format(dist))
        missing_range = consensus_file.split(".")[0].split("_")[0]
        replica_no = consensus_file.split(".")[0].split("_")[1]
        csv_out_list.append([replica_no, missing_range, dist])
# print(csv_out_list)

Removing numbers from consensus trees
Processing file: terrace_output/consensus/14-17_12_gt_species.raxml.consensusTreeMRE
RF distance: 0.35294117647058826


In [36]:
stats_file = "stats/RF_consensus.csv"
with open(stats_file, "w") as stat_file:
    csv.writer(stat_file).writerows(csv_out_list)